## Inisialisasi Library untuk Tahap Inference

Pada tahap inference atau prediksi, kita perlu memuat beberapa library penting yang dibutuhkan untuk membaca model, melakukan preprocessing input, dan mengatur eksekusi program. Berikut penjelasan masing-masing library:

- `pickle`: Digunakan untuk membuka file hasil serialisasi model dan tokenizer yang sebelumnya disimpan dalam format `.pkl`.
- `numpy`: Digunakan untuk manipulasi data numerik seperti array, yang umum digunakan dalam pengolahan data machine learning.
- `tensorflow`: Digunakan untuk memuat dan menjalankan model deep learning, khususnya model LSTM.
- `pad_sequences` dari Keras: Digunakan untuk menyamakan panjang input teks agar dapat diproses oleh model LSTM.
- `warnings`: Digunakan untuk menonaktifkan peringatan runtime agar output lebih bersih dan mudah dibaca.

In [1]:
# Mengimpor library untuk membaca file model dan tokenizer yang disimpan
import pickle

# Mengimpor library untuk manipulasi array dan operasi numerik
import numpy as np

# Mengimpor TensorFlow untuk memuat dan menjalankan model deep learning
import tensorflow as tf

# Mengimpor fungsi untuk padding input teks agar memiliki panjang yang seragam
from keras.preprocessing.sequence import pad_sequences

# Menonaktifkan peringatan agar output lebih bersih
import warnings
warnings.filterwarnings("ignore")

import os

## Memuat Model dan Alat Preprocessing untuk Inference

Pada tahap ini, kita akan memuat ulang seluruh model serta alat bantu preprocessing yang telah disimpan di dalam folder `Models/`.  
Tahapan ini penting dilakukan sebelum melakukan inference (prediksi) pada data baru.

#### Komponen yang dimuat:

- **Model LSTM**: File `.h5` berisi model neural network LSTM untuk klasifikasi sentimen berbasis word embedding.
- **Model SVM**: File `.pkl` hasil serialisasi model Support Vector Machine yang telah dilatih.
- **Model Random Forest**: File `.pkl` yang berisi model machine learning Random Forest.
- **TF-IDF Vectorizer**: Digunakan untuk mentransformasi teks menjadi fitur frekuensi kata.
- **Tokenizer**: Digunakan untuk mengubah teks menjadi urutan token angka untuk model LSTM.

Semua file ini disimpan di dalam folder `Models/` dan dibaca ke dalam memori agar siap digunakan untuk proses klasifikasi data baru.

In [2]:
# Membuat folder baru bernama 'Models'
os.makedirs('Models', exist_ok=True)

In [3]:
lstm_model = tf.keras.models.load_model("Models/lstm_sentiment_model.h5")

with open("Models/svm_sentiment_model.pkl", "rb") as f:
    svm_model = pickle.load(f)

with open("Models/rf_sentiment_model.pkl", "rb") as f:
    rf_model = pickle.load(f)

with open("Models/vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

with open("Models/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

## Preprocessing Untuk Data Baru

Pada tahap ini, kita menyiapkan sejumlah data teks baru yang akan digunakan untuk pengujian model klasifikasi sentimen yang telah dilatih sebelumnya.

In [4]:
# Data komentar baru yang digunakan untuk prediksi
komentar = [
    "Aplikasi ini sangat membantu dalam cek deteksi spam!",  #positif
    "Aplikasinya lemot banget",  #negatif
    "Kalau mau lihat tag harus premium, tapi harga langganannya murah kok"  #netral
]

# Preprocessing untuk Model LSTM:
new_sequences = tokenizer.texts_to_sequences(komentar)

# padding pada urutan token dengan maxlen=100
new_padded = pad_sequences(new_sequences, maxlen=100)

# Preprocessing untuk Model SVM dan Random Forest:
new_tfidf = vectorizer.transform(komentar)

## Perform Sentiment Prediction

Llangkah selanjutnya adalah melakukan prediksi sentimen menggunakan tiga model berbeda yang telah dilatih sebelumnya, yaitu LSTM, SVM, dan Random Forest.


In [5]:
# Melakukan prediksi dengan model LSTM
lstm_preds = np.argmax(lstm_model.predict(new_padded), axis=1)

# Melakukan prediksi dengan model SVM
svm_preds = svm_model.predict(new_tfidf)

# Melakukan prediksi dengan model Random Forest
rf_preds = rf_model.predict(new_tfidf)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 552ms/step


## Menampilkan Hasil Prediksi

Setelah proses prediksi dilakukan, tahap selanjutnya adalah menampilkan hasil prediksi sentimen dari ketiga model yaitu LSTM, SVM, dan Random Forest.

In [6]:
# Mapping label numerik ke label string
label_map = {0: "negative", 1: "neutral", 2: "positive"}

# Menampilkan hasil prediksi
print("\nHasil Prediksi : ")
for i, text in enumerate(komentar):
    print(f"Komentar : {text}")
    print(f"LSTM Prediction: {label_map[lstm_preds[i]]}")  # Hasil prediksi model LSTM
    print(f"SVM Prediction: {label_map[svm_preds[i]]}")    # Hasil prediksi model SVM
    print(f"Random Forest Prediction: {label_map[rf_preds[i]]}")  # Hasil prediksi model RF
    print("-" * 100)


Hasil Prediksi : 
Komentar : Aplikasi ini sangat membantu dalam cek deteksi spam!
LSTM Prediction: positive
SVM Prediction: positive
Random Forest Prediction: positive
----------------------------------------------------------------------------------------------------
Komentar : Aplikasinya lemot banget
LSTM Prediction: negative
SVM Prediction: negative
Random Forest Prediction: negative
----------------------------------------------------------------------------------------------------
Komentar : Kalau mau lihat tag harus premium, tapi harga langganannya murah kok
LSTM Prediction: neutral
SVM Prediction: neutral
Random Forest Prediction: neutral
----------------------------------------------------------------------------------------------------
